<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment%2010/Assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install optuna

     |████████████████████████████████| 296kB 10.6MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 1.1MB 10.0MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 143kB 36.1MB/s 
     |████████████████████████████████| 112kB 36.7MB/s 
     |████████████████████████████████| 481kB 33.9MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=753808f351582954e68e4896a79badfd03a2e61ae2211744a4a7355a96483e0c
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=a53744d922078379af7facc24e22ea05f90b9b7c939b0863a0167d02999897eb
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sh

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import torch.optim as optim
import optuna
from optuna.pruners import ThresholdPruner

In [22]:
transforms = transforms.Compose([
        transforms.ToTensor()
    ])
data_dir='/content/data'

train_ds = torchvision.datasets.FashionMNIST(root = data_dir, train=True, download=True, transform=transforms)
val_ds = torchvision.datasets.FashionMNIST(root = data_dir, train=False, download=True, transform=transforms)

In [23]:
loss_function = nn.CrossEntropyLoss()

In [24]:
class Net(nn.Module): 
  def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 20, 5, 1) 
      self.conv2 = nn.Conv2d(20, 50, 5, 1) 
      self.fc1 = nn.Linear(4*4*50, 500) 
      self.fc2 = nn.Linear(500, 10)

  def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.max_pool2d(x, 2, 2)
      x = F.relu(self.conv2(x))
      x = F.max_pool2d(x, 2, 2)
      x = x.view(-1, 4*4*50)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return F.log_softmax(x, dim=1)

In [25]:
def train(trial,epochs,optimizer,train_loader,cnn,device):
    epoch_loss = []
    epoch_acc = []
    for epoch in range(epochs):
        running_loss = []
        running_acc = []
        for local_batch, local_labels in train_loader:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            optimizer.zero_grad()
            outputs = cnn(local_batch)
            loss = loss_function(outputs, local_labels)
            loss.backward()
            optimizer.step()
            running_loss += [loss.item()]
            _, predicted = torch.max(outputs, 1)
            running_acc += [(predicted == local_labels).sum().item()/train_loader.batch_size]
        
        epoch_acc.append(np.mean(running_acc))
        epoch_loss.append(np.mean(running_loss))
        print('[{:d}] loss: {:7.5f} - acc {:7.5f}'.format(epoch + 1,epoch_loss[-1],epoch_acc[-1]))
        if len(epoch_loss) > 4:
            print('var: ',np.var(epoch_loss[-5:]))
            trial.report(np.var(epoch_loss[-5:]), epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()
                
    return epoch_loss[-1]

In [26]:
opt_func = [optim.SGD, optim.Adam, optim.RMSprop]
def objective(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    print(device)
    torch.backends.cudnn.benchmark = True
    opt_id = trial.suggest_categorical("optimizer",range(len(opt_func)))
    optim.Adam
    batch_sz = 2**trial.suggest_int('batch_size',4,6,1)

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_sz, shuffle=True)
    cnn = Net()
    cnn.to(device)
    optimizer= opt_func[opt_id](cnn.parameters(),lr=0.01)
    print(opt_func[opt_id])
    intermediate_value=train(trial,
                             trial.suggest_int('num_epoch',30,70,20),
                             optimizer,
                             train_loader,
                             cnn,
                             device)

    return intermediate_value

In [27]:
study = optuna.create_study(pruner=ThresholdPruner(lower=0.01))
study.optimize(objective, n_trials=50)

[I 2021-02-05 09:07:12,452] A new study created in memory with name: no-name-b4f402f2-654f-48ec-a898-8ad18cc28b28


cuda:0
<class 'torch.optim.sgd.SGD'>
[1] loss: 0.94100 - acc 0.65582
[2] loss: 0.57559 - acc 0.78667
[3] loss: 0.48434 - acc 0.82242
[4] loss: 0.43360 - acc 0.84260
[5] loss: 0.39815 - acc 0.85595
var:  0.03861727310046161


[I 2021-02-05 09:08:26,931] Trial 0 pruned. 


[6] loss: 0.37186 - acc 0.86582
var:  0.005195595892725764
cuda:0
<class 'torch.optim.sgd.SGD'>
[1] loss: 0.83012 - acc 0.70070
[2] loss: 0.48050 - acc 0.82260
[3] loss: 0.40847 - acc 0.85130
[4] loss: 0.36606 - acc 0.86663
[5] loss: 0.33568 - acc 0.87775
var:  0.03228454197313356


[I 2021-02-05 09:10:13,670] Trial 1 pruned. 


...

[6] loss: 0.49666 - acc 0.82130
var:  2.9641329663658462e-05
cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.08622 - acc 0.76457
[2] loss: 0.48624 - acc 0.82397
[3] loss: 0.48290 - acc 0.82552
[4] loss: 0.49521 - acc 0.82333
[5] loss: 0.48946 - acc 0.82548
var:  0.4084751141207638


[I 2021-02-05 10:33:41,325] Trial 48 pruned. 


[6] loss: 0.49424 - acc 0.82615
var:  2.1848217512171677e-05
cuda:0
<class 'torch.optim.rmsprop.RMSprop'>
[1] loss: 2.61477 - acc 0.79260
[2] loss: 0.49084 - acc 0.82298
[3] loss: 0.48649 - acc 0.83005
[4] loss: 0.48921 - acc 0.82855
[5] loss: 0.48950 - acc 0.82730
var:  0.7230156621077786


[I 2021-02-05 10:35:48,678] Trial 49 pruned. 


[6] loss: 0.49285 - acc 0.82888
var:  4.358804284210479e-06


In [28]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_num_epoch,params_optimizer,state
0,0,0.005196,2021-02-05 09:07:12.456076,2021-02-05 09:08:26.931179,0 days 00:01:14.475103,5,30,0,PRUNED
1,1,0.003432,2021-02-05 09:08:26.932154,2021-02-05 09:10:13.670581,0 days 00:01:46.738427,4,30,0,PRUNED
2,2,0.005309,2021-02-05 09:10:13.671719,2021-02-05 09:11:28.200186,0 days 00:01:14.528467,5,70,0,PRUNED
3,3,0.001997,2021-02-05 09:11:28.201229,2021-02-05 09:13:17.712347,0 days 00:01:49.511118,4,30,1,PRUNED
4,4,0.003361,2021-02-05 09:13:17.713588,2021-02-05 09:15:04.176130,0 days 00:01:46.462542,4,70,0,PRUNED
5,5,0.003608,2021-02-05 09:15:04.177345,2021-02-05 09:16:50.773815,0 days 00:01:46.596470,4,70,0,PRUNED
6,6,0.000677,2021-02-05 09:16:50.774945,2021-02-05 09:17:52.853416,0 days 00:01:02.078471,6,50,2,PRUNED
7,7,0.005115,2021-02-05 09:17:52.855038,2021-02-05 09:19:06.700644,0 days 00:01:13.845606,5,50,0,PRUNED
8,8,0.000034,2021-02-05 09:19:06.701781,2021-02-05 09:21:11.151137,0 days 00:02:04.449356,4,30,2,PRUNED
9,9,0.003320,2021-02-05 09:21:11.152208,2021-02-05 09:22:58.075405,0 days 00:01:46.923197,4,50,0,PRUNED
